# cropping.py

In [17]:
import cv2
import glob, os


In [18]:
negDir = 'data.row/images_cropped/neg/'
posDir = 'data.row/images_cropped/pos/'
imgDim = 224  

In [23]:
for imgFile in glob.iglob('data.row/image/**/**/*.png', recursive=True):
    #print(imgFile)
    imgID = os.path.basename(imgFile).split(".png")[0]
    img = cv2.imread(imgFile)
    grayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #Thresholding
    th, threshed = cv2.threshold(grayImg, 0, 255, cv2.THRESH_BINARY)
    #Find the max-area contour
    contours, hierarchy = cv2.findContours(threshed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = sorted(contours, key=cv2.contourArea)[-1]
    x, y, w, h = cv2.boundingRect(cnt)
    imgCropped = img[y:y + h, x:x + w]#Cropped image
    
    imgCropped = cv2.resize(imgCropped, (imgDim, imgDim))#resize
    
    #Saving cropped image
    if 'TS' in imgFile:
        cv2.imwrite(posDir + imgID + '.png', imgCropped, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
    elif 'TC'in imgFile:
        cv2.imwrite(negDir + imgID + '.png', imgCropped, [int(cv2.IMWRITE_JPEG_QUALITY), 100])


# imgsNPZ.py

In [40]:
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import glob

In [41]:
imgDim = 256
colorMode = "grayscale"
colorChannels = 1

In [46]:
imgDir = 'data.row/images_cropped/**/*.png'
outputFile = 'data.row/ASD.npz'

In [47]:
imgCount = 0
for file in glob.iglob(imgDir, recursive=True):
    imgCount = imgCount + 1
print("Number of images:", imgCount)

Number of images: 3103


In [48]:
X = np.zeros((imgCount, imgDim, imgDim, colorChannels), dtype=np.uint8)
y = np.zeros((imgCount, 1), dtype=np.uint8)

In [49]:
for i, imgFile in enumerate(glob.iglob(imgDir, recursive=True)):
    #print(imgFile)
    img = load_img(imgFile, color_mode=colorMode, target_size=(imgDim, imgDim))
    img = img_to_array(img)
    X[i] = img

    label = None
    if "neg" in imgFile:
        label = 0
    elif "pos" in imgFile:
        label = 1

    y[i] = label

print("X shape:", X.shape)
print("y shape:", y.shape)

X = (X / 255.0) #Normalization

#Saving all into compressed Numpy array
np.savez_compressed(outputFile, X=X, y=y)

X shape: (3103, 256, 256, 1)
y shape: (3103, 1)
